In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('steam_games.csv.zip', compression = 'zip')

In [3]:
clean = data.drop(['url', 'all_reviews', 'recent_reviews', 'game_description', 'recommended_requirements', 'publisher', 'desc_snippet', 'developer', 'release_date', 'types', 'popular_tags', 'minimum_requirements'], axis = 1).copy()

In [4]:
# total comments
clean['totalComments'] = data['all_reviews'].str.split(',', n = 1, expand = True)[1]
clean['totalComments'] = clean['totalComments'].str.replace('\).*', '').str.replace('\(', '').str.replace(',', '').str.replace('- Need more user reviews to generate a score', '10')

In [5]:
# postive proportion
clean['posProp'] = data['all_reviews'].str.split('- ', n = 1, expand = True)[1].str[0:2].str.replace('%', '').str.replace('Ne', '51')
clean.drop(clean[clean['posProp'].isna()].index, axis = 0, inplace = True)

In [6]:
# free
clean['original_price'] = clean['original_price'].str.replace('$', '')
clean.original_price = pd.to_numeric(clean.original_price, errors='coerce')
values = {'original_price': 'Free'}
clean.fillna(value=values, inplace = True)
clean['free'] = clean['original_price'] == 'Free'

In [7]:
# discount
clean['discount_price'] = clean['discount_price'].str.replace('$', '')
clean.discount_price = pd.to_numeric(clean.discount_price, errors='coerce')
values = {'discount_price': 0}
clean.fillna(value=values, inplace = True)

In [8]:
# original price
clean['original_price'] = clean['original_price'].replace('Free', 0.1)
clean['discount'] = (clean['original_price'] - clean['discount_price']) / clean['original_price']
clean['discount'] = clean['discount'].replace(1, 0)
negSlice = clean[clean['discount'] < 0].copy()
negSlice['discount'] = 0
clean.drop(clean[clean['discount'] < 0].index, axis = 0, inplace = True)
clean = clean.append(negSlice, ignore_index = True).reset_index(drop = True)
clean.drop(['discount_price'], axis = 1, inplace = True)
clean['original_price'] = clean['original_price'].replace(0.1, 0)

In [9]:
# mature content
values = {'mature_content': 'zero'}
clean.fillna(value=values, inplace = True)
clean['mature_content'] = clean['mature_content'].str.contains("sex[y|ual]|violen[t|ce]|blood|scary|kill|sho[o]t|murder|nudity|sensitive|not appropriate|expos[e|ure]|unsuitable|horror|mature|erotic|boob|breast|chest|gore", case=False, regex=True)

In [10]:
# achievement
values = {'achievements': 0}
clean.fillna(value=values, inplace = True)
clean['achievements'] = clean['achievements'].astype('int64')

In [11]:
# game details
values = {'game_details': 'no'}
clean.fillna(value=values, inplace = True)
clean['multi_player'] = clean['game_details'].str.contains('Multi-player', case = False)
clean['trading_cards'] = clean['game_details'].str.contains('Steam Trading Cards', case = False)
clean['online'] = clean['game_details'].str.contains('Online', case = False)
clean['controller'] = clean['game_details'].str.contains('Controller', case = False)
clean['cloud'] = clean['game_details'].str.contains('Cloud', case = False)
clean.drop('game_details', axis = 1, inplace = True)

In [12]:
# languages
clean.drop(clean[clean['languages'].isnull()].index, axis = 0, inplace = True)
clean['totalLan'] = clean['languages'].str.count(',') + 1
clean['English'] = clean['languages'].str.contains('English')
clean['Chinese'] = clean['languages'].str.contains('Simplified Chinese')
clean['Russian'] = clean['languages'].str.contains('Russian')
clean['Spanish'] = clean['languages'].str.contains('Spanish - Spain')
clean['Portuguese'] = clean['languages'].str.contains('Portuguese - Brazil')
clean['German'] = clean['languages'].str.contains('German')
clean.drop('languages', axis = 1, inplace = True)

In [13]:
# genre
values = {'genre': 'Action,'}
clean.fillna(value=values, inplace = True)

In [14]:
action = clean[clean['genre'].str.contains('Action', case = False)].copy()
action['genre'] = action['genre'].str.replace(".*", 'Action')
clean.drop(action.index, inplace = True)
adventure = clean[clean['genre'].str.contains('Adventure', case = False)].copy()
adventure['genre'] = adventure['genre'].str.replace(".*", 'Adventure')
clean.drop(adventure.index, inplace = True)
strategy = clean[clean['genre'].str.contains('Strategy', case = False)].copy()
strategy['genre'] = strategy['genre'].str.replace(".*", 'Strategy')
clean.drop(strategy.index, inplace = True)
casual = clean[clean['genre'].str.contains('Casual', case = False)].copy()
casual['genre'] = casual['genre'].str.replace(".*", 'Casual')
clean.drop(casual.index, inplace = True)
simulation = clean[clean['genre'].str.contains('Simulation', case = False)].copy()
simulation['genre'] = simulation['genre'].str.replace(".*", 'Simulation')
clean.drop(simulation.index, inplace = True)
rpg = clean[clean['genre'].str.contains('RPG', case = False)].copy()
rpg['genre'] = rpg['genre'].str.replace(".*", 'RPG')
clean.drop(rpg.index, inplace = True)
other = clean.copy()
other['genre'] = other['genre'].str.replace(".*", 'Other')
clean.drop(other.index, inplace = True)

In [15]:
clean = clean.append(action, ignore_index = False)
clean = clean.append(adventure, ignore_index = False)
clean = clean.append(strategy, ignore_index = False)
clean = clean.append(casual, ignore_index = False)
clean = clean.append(simulation, ignore_index = False)
clean = clean.append(rpg, ignore_index = False)
clean = clean.append(other, ignore_index = False)

In [16]:
clean['genre'] = clean['genre'].str.replace('ActionAction', 'Action')
clean['genre'] = clean['genre'].str.replace('AdventureAdventure', 'Adventure')
clean['genre'] = clean['genre'].str.replace('StrategyStrategy', 'Strategy')
clean['genre'] = clean['genre'].str.replace('CasualCasual', 'Casual')
clean['genre'] = clean['genre'].str.replace('SimulationSimulation', 'Simulation')
clean['genre'] = clean['genre'].str.replace('RPGRPG', 'RPG')
clean['genre'] = clean['genre'].str.replace('OtherOther', 'Other')

In [17]:
clean['posProp'] = clean['posProp'].astype('int64')
clean['review'] = clean['posProp'] >= 50

In [18]:
#IQR = clean['achievements'].quantile(q = 0.75) - clean['achievements'].quantile(q = 0.25)
#mean = clean['achievements'].mean()
#upper = mean + 1.5 * IQR
#lower = mean - 1.5 * IQR
#index_names = clean[(clean['achievements'] > upper) | (clean['achievements'] < lower)].index
#clean.drop(index_names, inplace = True)

In [19]:
IQR = clean['original_price'].quantile(q = 0.75) - clean['original_price'].quantile(q = 0.25)
mean = clean['original_price'].mean()
upper = mean + 1.5 * IQR
lower = mean - 1.5 * IQR
index_names = clean[(clean['original_price'] > upper)].index
clean.drop(index_names, inplace = True)

In [20]:
#IQR = clean['totalLan'].quantile(q = 0.75) - clean['totalLan'].quantile(q = 0.25)
#mean = clean['totalLan'].mean()
#upper = mean + 1.5 * IQR
#lower = mean - 1.5 * IQR
#index_names = clean[(clean['totalLan'] > upper) | (clean['totalLan'] < lower)].index
#clean.drop(index_names, inplace = True)

In [21]:
clean['totalComments'] = clean['totalComments'].astype('int64')
clean['discount'] = clean['discount'].astype('float64')

In [22]:
highDiscount = clean[clean['discount'] >= 0.5].copy()
lowDiscount = clean[(clean['discount'] < 0.5) & (clean['discount'] > 0)].copy()
noDiscount = clean[clean['discount'] == 0].copy()
lowDiscount.loc[lowDiscount['discount'] < 0.5, 'discount'] = "Low"
highDiscount.loc[highDiscount['discount'] >= 0.5, 'discount'] = "High"
noDiscount.loc[noDiscount['discount'] == 0, 'discount'] = "Zero"
clean = highDiscount.append(lowDiscount, ignore_index = True)
clean = clean.append(noDiscount, ignore_index = True).reset_index(drop = True)

In [23]:
clean.sort_values(by = 'review', inplace = True)

In [24]:
clean.to_csv('review.csv', index=False)